**This notebook is an exercise in the [SQL](https://www.kaggle.com/learn/intro-to-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/dansbecker/as-with).**

---


# Introduction

You are getting to the point where you can own an analysis from beginning to end. So you'll do more data exploration in this exercise than you've done before.  Before you get started, run the following set-up code as usual. 

In [2]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex5 import *
print("Setup Complete")

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


Setup Complete


You'll work with a dataset about taxi trips in the city of Chicago. Run the cell below to fetch the `chicago_taxi_trips` dataset.

In [3]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

Using Kaggle's public dataset BigQuery integration.


# Exercises

You are curious how much slower traffic moves when traffic volume is high. This involves a few steps.

### 1) Find the data
Before you can access the data, you need to find the table name with the data.

*Hint*: Tab completion is helpful whenever you can't remember a command. Type `client.` and then hit the tab key. Don't forget the period before hitting tab.

In [4]:
# Your code here to find the table name
tables = client.list_tables(dataset)

for table in tables:
    print(table.table_id)
    print(table.labels)


# # Attempt at printing it directly
# tables2 = client.list_tables(dataset)
# print((*tables2).table_id)

taxi_trips
{}


In [5]:
help(client.list_tables)

Help on method list_tables in module google.cloud.bigquery.client:

list_tables(dataset, max_results=None, page_token=None, retry=<google.api_core.retry.Retry object at 0x7fd9e9fee390>, timeout=None) method of kaggle_gcp.PublicBigqueryClient instance
    List tables in the dataset.
    
    See
    https://cloud.google.com/bigquery/docs/reference/rest/v2/tables/list
    
    Args:
        dataset (Union[                 google.cloud.bigquery.dataset.Dataset,                 google.cloud.bigquery.dataset.DatasetReference,                 str,             ]):
            A reference to the dataset whose tables to list from the
            BigQuery API. If a string is passed in, this method attempts
            to create a dataset reference from a string using
            :func:`google.cloud.bigquery.dataset.DatasetReference.from_string`.
        max_results (Optional[int]):
            Maximum number of tables to return. If not passed, defaults
            to a value set by the API.
    

Notice that it return an **iterator**. Can only be printed stepwise, not directly

In [6]:
# Write the table name as a string below
table_name = "taxi_trips"

# Check your answer
q_1.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For the solution, uncomment the line below.

In [7]:
# q_1.solution()

### 2) Peek at the data

Use the next code cell to peek at the top few rows of the data. Inspect the data and see if any issues with data quality are immediately obvious. 

**Your code here**


In [8]:
help(client.get_table)

Help on method get_table in module google.cloud.bigquery.client:

get_table(table, retry=<google.api_core.retry.Retry object at 0x7fd9e9fee390>, timeout=None) method of kaggle_gcp.PublicBigqueryClient instance
    Fetch the table referenced by ``table``.
    
    Args:
        table (Union[                 google.cloud.bigquery.table.Table,                 google.cloud.bigquery.table.TableReference,                 str,             ]):
            A reference to the table to fetch from the BigQuery API.
            If a string is passed in, this method attempts to create a
            table reference from a string using
            :func:`google.cloud.bigquery.table.TableReference.from_string`.
        retry (Optional[google.api_core.retry.Retry]):
            How to retry the RPC.
        timeout (Optional[float]):
            The number of seconds to wait for the underlying HTTP transport
            before using ``retry``.
    
    Returns:
        google.cloud.bigquery.table.Table:

In [9]:
help(client.list_rows)

Help on method list_rows in module google.cloud.bigquery.client:

list_rows(table, selected_fields=None, max_results=None, page_token=None, start_index=None, page_size=None, retry=<google.api_core.retry.Retry object at 0x7fd9e9fee390>, timeout=None) method of kaggle_gcp.PublicBigqueryClient instance
    List the rows of the table.
    
    See
    https://cloud.google.com/bigquery/docs/reference/rest/v2/tabledata/list
    
    .. note::
    
       This method assumes that the provided schema is up-to-date with the
       schema as defined on the back-end: if the two schemas are not
       identical, the values returned may be incomplete. To ensure that the
       local copy of the schema is up-to-date, call ``client.get_table``.
    
    Args:
        table (Union[                 google.cloud.bigquery.table.Table,                 google.cloud.bigquery.table.TableListItem,                 google.cloud.bigquery.table.TableReference,                 str,             ]):
            The 

See https://cloud.google.com/bigquery/docs/reference/libraries#client-libraries-install-python for a quick overview

In [27]:
table_id = table.full_table_id
table_id

'bigquery-public-data:chicago_taxi_trips.taxi_trips'

In [11]:
table = client.get_table(table_id) # Make an API request

# View table properties
print(
    "Got table '{}.{}.{}'.".format(table.project, table.dataset_id, table.table_id)
)
print("Table schema: {}".format(table.schema))
print("Table description: {}".format(table.description))
print("Table has {} rows".format(table.num_rows))

ValueError: table_id must be a fully-qualified ID in standard SQL format, e.g., "project.dataset.table_id", got taxi_trips

In [12]:
table_ref = dataset_ref.table("taxi_trips")
print(table_ref)

table = client.get_table(table_ref)

# View table properties
print(
    "Got table '{}.{}.{}'.".format(table.project, table.dataset_id, table.table_id)
)
print("Table schema: {}".format(table.schema))
print("Table description: {}".format(table.description))
print("Table has {} rows".format(table.num_rows))

TableReference(DatasetReference('bigquery-public-data', 'chicago_taxi_trips'), 'taxi_trips')
Got table 'bigquery-public-data.chicago_taxi_trips.taxi_trips'.
Table schema: [SchemaField('unique_key', 'STRING', 'REQUIRED', 'Unique identifier for the trip.', (), None), SchemaField('taxi_id', 'STRING', 'REQUIRED', 'A unique identifier for the taxi.', (), None), SchemaField('trip_start_timestamp', 'TIMESTAMP', 'NULLABLE', 'When the trip started, rounded to the nearest 15 minutes.', (), None), SchemaField('trip_end_timestamp', 'TIMESTAMP', 'NULLABLE', 'When the trip ended, rounded to the nearest 15 minutes.', (), None), SchemaField('trip_seconds', 'INTEGER', 'NULLABLE', 'Time of the trip in seconds.', (), None), SchemaField('trip_miles', 'FLOAT', 'NULLABLE', 'Distance of the trip in miles.', (), None), SchemaField('pickup_census_tract', 'INTEGER', 'NULLABLE', 'The Census Tract where the trip began. For privacy, this Census Tract is not shown for some trips.', (), None), SchemaField('dropoff_c

In [13]:
print(*table.schema, sep='\n\n\n')

SchemaField('unique_key', 'STRING', 'REQUIRED', 'Unique identifier for the trip.', (), None)


SchemaField('taxi_id', 'STRING', 'REQUIRED', 'A unique identifier for the taxi.', (), None)


SchemaField('trip_start_timestamp', 'TIMESTAMP', 'NULLABLE', 'When the trip started, rounded to the nearest 15 minutes.', (), None)


SchemaField('trip_end_timestamp', 'TIMESTAMP', 'NULLABLE', 'When the trip ended, rounded to the nearest 15 minutes.', (), None)


SchemaField('trip_seconds', 'INTEGER', 'NULLABLE', 'Time of the trip in seconds.', (), None)


SchemaField('trip_miles', 'FLOAT', 'NULLABLE', 'Distance of the trip in miles.', (), None)


SchemaField('pickup_census_tract', 'INTEGER', 'NULLABLE', 'The Census Tract where the trip began. For privacy, this Census Tract is not shown for some trips.', (), None)


SchemaField('dropoff_census_tract', 'INTEGER', 'NULLABLE', 'The Census Tract where the trip ended. For privacy, this Census Tract is not shown for some trips.', (), None)


SchemaField('p

In [15]:
help(client.list_rows)

Help on method list_rows in module google.cloud.bigquery.client:

list_rows(table, selected_fields=None, max_results=None, page_token=None, start_index=None, page_size=None, retry=<google.api_core.retry.Retry object at 0x7fd9e9fee390>, timeout=None) method of kaggle_gcp.PublicBigqueryClient instance
    List the rows of the table.
    
    See
    https://cloud.google.com/bigquery/docs/reference/rest/v2/tabledata/list
    
    .. note::
    
       This method assumes that the provided schema is up-to-date with the
       schema as defined on the back-end: if the two schemas are not
       identical, the values returned may be incomplete. To ensure that the
       local copy of the schema is up-to-date, call ``client.get_table``.
    
    Args:
        table (Union[                 google.cloud.bigquery.table.Table,                 google.cloud.bigquery.table.TableListItem,                 google.cloud.bigquery.table.TableReference,                 str,             ]):
            The 

Output is an **iterator**.

In [16]:
client.list_rows(table, max_results=10).to_dataframe()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  """Entry point for launching an IPython kernel.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,3190243aa353d190bf2e5f366cd61617fa96f6a8,9d4a0cf00e9283302f40800a5da6e7017db07876ff895d...,2015-08-26 11:15:00+00:00,2015-08-26 11:15:00+00:00,0,0.0,NaN,NaN,NaN,NaN,...,0.0,7.31,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,NaN,NaN,None,NaN,NaN,None
1,6d5b6e5f3e5397f08e770beeb5f34433b0491d12,c2ca667849d5b9c2d10a15bf7e6a6ab358ab7a98cce37b...,2014-08-01 21:45:00+00:00,2014-08-01 21:45:00+00:00,0,0.0,NaN,NaN,NaN,NaN,...,0.0,8.45,Credit Card,T.A.S. - Payment Only,NaN,NaN,None,NaN,NaN,None
2,980c986313ec13a9d59412826c83bfa65e01d0c3,60e9b32a85d0045d670d329891f51b9796543659769e9a...,2014-08-01 18:45:00+00:00,2014-08-01 18:45:00+00:00,0,0.0,NaN,NaN,NaN,NaN,...,0.0,8.05,Credit Card,T.A.S. - Payment Only,NaN,NaN,None,NaN,NaN,None
3,29aea409062ff5c29c33fecff902acd8e58e5e0f,73f6888d8d8f9808f6624377edfa1e2a0a574b06eafbc9...,2014-07-31 09:45:00+00:00,2014-07-31 09:45:00+00:00,0,0.0,NaN,NaN,NaN,NaN,...,0.0,12.06,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,NaN,NaN,None,NaN,NaN,None
4,e83f8835694883f25c11fc70bdf8443b5c68448f,55af2e70fd4624a43fea70121974c54db44c8e7582ce3d...,2014-07-16 08:45:00+00:00,2014-07-16 08:45:00+00:00,0,0.0,NaN,NaN,NaN,NaN,...,1.0,4.25,Cash,Blue Ribbon Taxi Association Inc.,NaN,NaN,None,NaN,NaN,None
5,c12f881d98be668fadbc3e7e44617c8753375fba,8b14b1329b11561a614d568ad4e470a1a9b7ea5dccce54...,2014-07-11 12:45:00+00:00,2014-07-11 12:45:00+00:00,0,0.0,NaN,NaN,NaN,NaN,...,0.0,51.95,Credit Card,T.A.S. - Payment Only,NaN,NaN,None,NaN,NaN,None
6,928e379eb1f37fa270884c6130062ba87d7cc81c,ff17670d96389440510dbf6c0dbfa442be6431b76d0d4c...,2014-08-01 18:15:00+00:00,2014-08-01 18:15:00+00:00,0,0.0,NaN,NaN,NaN,NaN,...,0.0,10.14,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,NaN,NaN,None,NaN,NaN,None
7,fce167dc5c2c37fdef18d58eeba1188abc1c1148,8f97e4651008e654e9adcc9b242245e8813c067d6f2559...,2014-07-16 08:45:00+00:00,2014-07-16 08:45:00+00:00,0,0.0,NaN,NaN,NaN,NaN,...,0.0,29.82,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,NaN,NaN,None,NaN,NaN,None
8,ffcf7ad0e1cf9d24f47cbb2ed5716e706e63c176,b98f8275bad36ea096224a8fc52c7de7e75cc9b7230074...,2014-08-01 21:15:00+00:00,2014-08-01 21:15:00+00:00,0,0.0,NaN,NaN,NaN,NaN,...,0.0,13.65,Credit Card,T.A.S. - Payment Only,NaN,NaN,None,NaN,NaN,None
9,92067684e144d1c403b307f2a350b913e1ea2612,ea7bd9f0e56d3a905d808f728cd0630e1c11d4f85a3d42...,2014-07-31 01:30:00+00:00,2014-07-31 01:30:00+00:00,0,0.0,NaN,NaN,NaN,NaN,...,0.0,18.45,Credit Card,T.A.S. - Payment Only,NaN,NaN,None,NaN,NaN,None


Table properties formating from : <https://cloud.google.com/bigquery/docs/tables#using_tables>

After deciding whether you see any important issues, run the code cell below.

In [17]:
# Check your answer (Run this code cell to receive credit!)
q_2.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
You can see the data by calling: 
```python
# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "taxi_trips" table
client.list_rows(table, max_results=5).to_dataframe()
```

Some trips in the top few rows have `trip_seconds` or `trip_miles` values of 0. 
Other location fields have values of `None`. That is a problem if we want to use those fields.


### 3) Determine when this data is from

If the data is sufficiently old, we might be careful before assuming the data is still relevant to traffic patterns today. Write a query that counts the number of trips in each year.  

Your results should have two columns:
- `year` - the year of the trips
- `num_trips` - the number of trips in that year

Hints:
- When using **GROUP BY** and **ORDER BY**, you should refer to the columns by the alias `year` that you set at the top of the **SELECT** query.
- The SQL code to **SELECT** the year from `trip_start_timestamp` is <code>SELECT EXTRACT(YEAR FROM trip_start_timestamp)</code>
- The **FROM** field can be a little tricky until you are used to it.  The format is:
    1. A backick (the symbol \`).
    2. The project name. In this case it is `bigquery-public-data`.
    3. A period.
    4. The dataset name. In this case, it is `chicago_taxi_trips`.
    5. A period.
    6. The table name. You used this as your answer in **1) Find the data**.
    7. A backtick (the symbol \`).

In [18]:
# Your code goes here
rides_per_year_query = """
                       SELECT EXTRACT(YEAR from trip_start_timestamp) AS year, COUNT(1) AS num_trips
                       FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                       GROUP BY year
                       ORDER BY year
                       """

# Set up the query (cancel the query if it would use too much of 
# your quota)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
rides_per_year_query_job = client.query(rides_per_year_query, job_config=safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
rides_per_year_result = rides_per_year_query_job.to_dataframe() # Your code goes here

# View results
print(rides_per_year_result)

# Check your answer
q_3.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


   year  num_trips
0  2013   27217716
1  2014   37395436
2  2015   32385875
3  2016   31759339
4  2017   24988003
5  2018   20732088
6  2019   16477365
7  2020    3889032
8  2021    3948019
9  2022         30


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For a hint or the solution, uncomment the appropriate line below.

In [19]:
#q_3.hint()
#q_3.solution()

### 4) Dive slightly deeper

You'd like to take a closer look at rides from 2017.  Copy the query you used above in `rides_per_year_query` into the cell below for `rides_per_month_query`.  Then modify it in two ways:
1. Use a **WHERE** clause to limit the query to data from 2017.
2. Modify the query to extract the month rather than the year.

In [21]:
# Your code goes here
rides_per_month_query = """
                       SELECT EXTRACT(MONTH from trip_start_timestamp) AS month, COUNT(1) AS num_trips
                       FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                       WHERE EXTRACT(YEAR from trip_start_timestamp) = 2017
                       GROUP BY month
                       ORDER BY month
                       """

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
rides_per_month_query_job = client.query(rides_per_month_query, job_config=safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
rides_per_month_result = rides_per_month_query_job.to_dataframe() # Your code goes here

# View results
print(rides_per_month_result)

# Check your answer
q_4.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


    month  num_trips
0       1    1972071
1       2    1909802
2       3    2362105
3       4    2194702
4       5    2323386
5       6    2324472
6       7    2054299
7       8    2079861
8       9    1950631
9      10    2141197
10     11    1907997
11     12    1767480


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For a hint or the solution, uncomment the appropriate line below.

In [22]:
#q_4.hint()
#q_4.solution()

### 5) Write the query

It's time to step up the sophistication of your queries.  Write a query that shows, for each hour of the day in the dataset, the corresponding number of trips and average speed.

Your results should have three columns:
- `hour_of_day` - sort by this column, which holds the result of extracting the hour from `trip_start_timestamp`.
- `num_trips` - the count of the total number of trips in each hour of the day (e.g. how many trips were started between 6AM and 7AM, independent of which day it occurred on).
- `avg_mph` - the average speed, measured in miles per hour, for trips that started in that hour of the day.  Average speed in miles per hour is calculated as `3600 * SUM(trip_miles) / SUM(trip_seconds)`. (The value 3600 is used to convert from seconds to hours.)

Restrict your query to data meeting the following criteria:
- a `trip_start_timestamp` between **2017-01-01** and **2017-07-01**
- `trip_seconds` > 0 and `trip_miles` > 0

You will use a common table expression (CTE) to select just the relevant rides.  Because this dataset is very big, this CTE should select only the columns you'll need to create the final output (though you won't actually create those in the CTE -- instead you'll create those in the later **SELECT** statement below the CTE).

This is a much harder query than anything you've written so far.  Good luck!

In [33]:
# Your code goes here
# speeds_query = """
#                WITH RelevantRides AS
#                (
#                    SELECT EXTRACT(HOUR FROM trip_start_timestamp) as hour_of_day, 
#                            3600 * (trip_miles) / (trip_seconds) AS mph
#                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
#                    WHERE EXTRACT(DATE FROM trip_start_timestamp) 
#                            BETWEEN '2017-01-01' AND '2017-07-01'
#                    AND trip_seconds > 0
#                    AND trip_miles > 0
#                )
#                SELECT hour_of_day,
#                        COUNT(1) AS num_trips, AVG(mph) as avg_mph
#                FROM RelevantRides
#                GROUP BY hour_of_day
#                ORDER BY hour_of_day
#                """

# speeds_query = """
#                WITH RelevantRides AS
#                (
#                    SELECT EXTRACT(HOUR FROM trip_start_timestamp) as hour_of_day, 
#                            3600 * (trip_miles) / (trip_seconds) AS mph
#                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
#                    WHERE trip_start_timestamp > '2017-01-01' 
#                    AND trip_start_timestamp < '2017-07-01'
#                    AND trip_seconds > 0
#                    AND trip_miles > 0
#                )
#                SELECT hour_of_day,
#                        COUNT(1) AS num_trips, AVG(mph) as avg_mph
#                FROM RelevantRides
#                GROUP BY hour_of_day
#                ORDER BY hour_of_day
#                """

speeds_query = """
               WITH RelevantRides AS
               (
                   SELECT EXTRACT(HOUR FROM trip_start_timestamp) AS hour_of_day, 
                          trip_miles, 
                          trip_seconds
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE trip_start_timestamp > '2017-01-01' AND 
                         trip_start_timestamp < '2017-07-01' AND 
                         trip_seconds > 0 AND 
                         trip_miles > 0
               )
               SELECT hour_of_day, 
                      COUNT(1) AS num_trips, 
                      3600 * SUM(trip_miles) / SUM(trip_seconds) AS avg_mph
               FROM RelevantRides
               GROUP BY hour_of_day
               ORDER BY hour_of_day
               """

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
speeds_query_job = client.query(speeds_query, job_config=safe_config) # Your code here

# API request - run the query, and return a pandas DataFrame
speeds_result = speeds_query_job.to_dataframe() # Your code here

# View results
print(speeds_result)

# Check your answer
q_5.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


    hour_of_day  num_trips    avg_mph
0             0     319339  20.230524
1             1     266529  18.937621
2             2     210147  18.777070
3             3     159668  20.158048
4             4     122183  26.736014
5             5     119312  30.769172
6             6     182738  24.588313
7             7     358406  17.735967
8             8     541775  15.079892
9             9     565548  16.543882
10           10     525120  18.539614
11           11     594603  18.928379
12           12     622324  17.838745
13           13     630181  17.671089
14           14     622465  16.974239
15           15     640430  15.688418
16           16     701435  14.283888
17           17     756627  12.462955
18           18     768251  13.646810
19           19     701064  16.642882
20           20     598614  19.536777
21           21     552726  20.433874
22           22     501095  19.531374
23           23     399587  19.877046


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

**NOTE**: The three queries return very different answers!!!

For the solution, uncomment the appropriate line below.

In [29]:
q_5.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

speeds_query = """
               WITH RelevantRides AS
               (
                   SELECT EXTRACT(HOUR FROM trip_start_timestamp) AS hour_of_day, 
                          trip_miles, 
                          trip_seconds
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE trip_start_timestamp > '2017-01-01' AND 
                         trip_start_timestamp < '2017-07-01' AND 
                         trip_seconds > 0 AND 
                         trip_miles > 0
               )
               SELECT hour_of_day, 
                      COUNT(1) AS num_trips, 
                      3600 * SUM(trip_miles) / SUM(trip_seconds) AS avg_mph
               FROM RelevantRides
               GROUP BY hour_of_day
               ORDER BY hour_of_day
               """

# Set up the query (cancel the query if it would use too much of 
# your quota)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
speeds_query_job = client.query(speeds_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
speeds_result = speeds_query_job.to_dataframe()

# View results
print(speeds_result)

```

That's a hard query. If you made good progress towards the solution, congratulations!

# Keep going

You can write very complex queries now with a single data source. But nothing expands the horizons of SQL as much as the ability to combine or **JOIN** tables.

**[Click here](https://www.kaggle.com/dansbecker/joining-data)** to start the last lesson in the Intro to SQL micro-course.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/intro-to-sql/discussion) to chat with other learners.*